In [1]:
import pandas as pd
import datetime as dt
import time

# The is where I stor the trading logic
from crossTrader import CrossTrader_v1

from starter import Start
from tester import Tester, run_test, sample_df
from plotingResult import Charts

Load the data and clean it:

In [2]:
directory = '/home/bk/Dropbox/scripts/git/Algorithmic-Trading/'
file = 'Binance_BTCUSDT_1h.csv'

# Firt row have a just a link ('https://www.CryptoDataDownload.com')
df = pd.read_csv(directory+file, skiprows=1)

# Clean the data a bit
df.index = df.date.map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %I-%p') if 'PM' in x or 'AM' in x else dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df = df.rename(columns=lambda x: x.capitalize())
df = df[['Open', 'High', 'Low', 'Close']].copy()

In [3]:
# Convert the data to 30 minuts time frame
df_30M = sample_df(df, '30Min')

# Convert the data to 5 minuts time frame
df_5M = sample_df(df, '5Min')

Set the parameters for our algorithum (The Start class is just a shortcut)

In [4]:
# The Start class just provide a dictionary with parameters for our trading alogrithim
start = Start()
start.start_data['sym'] = 'BTC'
start.start_data['Algo'] = 'cross'
start.start_data['Put real Trades'] = False
trading_logic, sym, tf, win, time_sleep, comments, put_real_trades, stop_method, limit_x, direction = start.get_parameters()
max_loss = False

# Since I type the win as str of 25,50 I need to convert it to int
fast_win, slow_win = [int(x) for x in win.split(',')] 

# Set the trading algoritum 
t1 = CrossTrader_v1(sym, tf, fast_win, slow_win)

# Set parameters
t1.trading_status = put_real_trades
if max_loss:
    t1.max_loss = max_loss
if limit_x:
    t1.limit_x = limit_x
t1.stop_method = stop_method

# Run ovre the function providing real live data with the get_data from the Tester class
# Thie is a functoin that provide data I am loading 
backtest = Tester(sym, tf)
t1.data_source.get_data = backtest.get_data
# make shure trades result will go to the Tester class to load latter
t1.record_trades = backtest.record_trades

{'Algo': 'cross',
 'Comment': '',
 'Direction': 'all',
 'Limit': False,
 'MA window': '25,50',
 'Put real Trades': False,
 'Sleep time': 5,
 'Stop Method': 'ma',
 'sym': 'BTC',
 'tf': '5m'}
Press Enter to continue, other key to change input j
Algo = cross bollinger
sym = BTC 
tf = 5m 
MA window = 25,50 50
Sleep time = 5 
Comment =  
Put real Trades = False 
Stop Method = ma bollinger
Stop Method = ma 
Limit = False 
Direction = all 


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# set the df, meaning the data we going to do our back test on.
backtest.df = df_5M.loc['2022']

In [ ]:
print ('%s  - Runing back test from %s to %s, %s data points' % (time.ctime(), backtest.df.index[0], backtest.df.index[-1], len(backtest.df)) )
# Run and check each data point as if its resived from a server, put trades and manage stops
backtest.reset()
run_test(t1, len(backtest.df))
result_df = backtest.join_result()
trades = result_df[result_df.pnl.notna()].copy()
print ('PnL sum: {:,.2f} |  Percentage of wining trades: {:.2%}, numebr of trades: {}'.format(trades.pnl.sum(), len(trades[trades.pnl > 0]) / len(trades), len(trades)))


In [ ]:
print ('%s  - Runing back test from %s to %s, %s data points' % (time.ctime(), backtest.df.index[0], backtest.df.index[-1], len(backtest.df)) )
# Run and check each data point as if its resived from a server, put trades and manage stops
backtest.reset()
run_test(t1, len(backtest.df))
result_df = backtest.join_result()
trades = result_df[result_df.pnl.notna()].copy()
print ('PnL sum: {:,.2f} |  Percentage of wining trades: {:.2%}, numebr of trades: {}'.format(trades.pnl.sum(), len(trades[trades.pnl > 0]) / len(trades), len(trades)))


In [ ]:
chart = Charts()

In [ ]:
chart.plot_trades(result_df)

In [ ]:
# reset the backtest result
backtest.reset()

# Change some of the trading parametrs
t1.limit = 2  # meanign close positoin once the porift x2 the stop

# Ran the backtest again
run_test(t1, len(backtest.df))
result_df = backtest.join_result()
trades = result_df[result_df.pnl.notna()]

In [ ]:
print ('PnL sum: {:,.2f} |  Percentage of wining trades: {:.2%}'.format(result_df.pnl.sum(), len(result_df[result_df.pnl > 0]) / len(result_df[result_df.pnl.notna()]) ))
print (len(trades))

In [ ]:
backtest.reset()

# Change some of the trading parametrs
t1.limit = 1  # meanign close positoin once the porift x2 the stop

run_test(t1, len(backtest.df))
result_df = backtest.join_result()
trades = result_df[result_df.pnl.notna()]

In [ ]:
print ('PnL sum: {:,.2f} |  Percentage of wining trades: {:.2%}'.format(result_df.pnl.sum(), len(result_df[result_df.pnl > 0]) / len(result_df[result_df.pnl.notna()]) ))
print (len(trades))